In [ ]:
#import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
import os
import sys
import matplotlib.pyplot as plt
import glob

In [ ]:
deeplabv3plus_dir="./src"
sys.path.append(deeplabv3plus_dir)

In [ ]:
gpu_options = tf.compat.v1.GPUOptions(visible_device_list="3", allow_growth=True)
config = tf.compat.v1.ConfigProto(gpu_options = gpu_options)
tf.compat.v1.enable_eager_execution(config=config)

In [ ]:
#from model import deeplab_v3plus
from data_utils import make_xy_from_data_paths, convert_y_to_image_array
from data_gen import DataGenerator
from label import Label

In [ ]:
#import importlib
#importlib.reload(sys.modules['DataGenerator'])

In [ ]:
traindata_dir = '../train_data'
#testdata_dir = '../kaigan_block/dataset/x_test2'

train_x_paths = glob.glob(os.path.join(traindata_dir,'*.jpg'))
train_x_paths.sort()
image_names = [os.path.basename(train_x_paths[i]).split('.')[0] for i in range(len(train_x_paths))]
train_y_paths=[]
for i, image_name in enumerate(image_names):
    p = os.path.join(traindata_dir, image_name+'.json')
    if os.path.exists(p):
        train_y_paths.append(p)
    else:
        train_y_paths.append(None)

In [ ]:
'''
seg_img_dir = "../pascal_voc_2012_datasets/VOCdevkit/VOC2012/SegmentationClass"
img_dir = "../pascal_voc_2012_datasets/VOCdevkit/VOC2012/JPEGImages"
train_set_path = "../pascal_voc_2012_datasets/VOCdevkit/VOC2012/ImageSets/Segmentation/train.txt"
valid_set_path = "../pascal_voc_2012_datasets/VOCdevkit/VOC2012/ImageSets/Segmentation/val.txt"

with open(train_set_path) as f:
    train_img_names = f.read().split("\n")[:-1]
with open(valid_set_path) as f:
    valid_img_names = f.read().split("\n")[:-1]

img_paths = [os.path.join(img_dir,train_img_names[i]) + ".jpg" for i in range(len(train_img_names))]
seg_img_paths = [os.path.join(seg_img_dir,train_img_names[i]) + ".png" for i in range(len(train_img_names))]

valid_x_paths = [os.path.join(img_dir,valid_img_names[i]) + ".jpg" for i in range(len(valid_img_names))]
valid_y_paths = [os.path.join(seg_img_dir,valid_img_names[i]) + ".png" for i in range(len(valid_img_names))]
#valid_x_paths = [os.path.join(img_dir,train_img_names[i]) + ".jpg" for i in range(len(train_img_names))][0:10]
#valid_y_paths = [os.path.join(seg_img_dir,train_img_names[i]) + ".png" for i in range(len(train_img_names))][0:10]
'''

In [ ]:
label_file_path = os.path.join(traindata_dir, 'label_list.csv')
label = Label(label_file_path)
image_size = (512,512)

In [ ]:
batch_size = 8
preprocess = keras.applications.xception.preprocess_input

In [ ]:
train_data_gen = DataGenerator(train_x_paths,
                               train_y_paths,
                               image_size,
                               label,
                               batch_size,
                               preprocess,
                               augmentation=True,
                               shuffle=True,
                               resize_or_crop="crop",
                               data_type="polygon")

In [ ]:
x,y0 = train_data_gen.__getitem__(1)
x = (x+1)/2
y =convert_y_to_image_array(y0, image_size, label)

i=3
plt.figure(figsize=(20,10))
plt.subplot(1,2,1)
plt.imshow(x[i,:,:,:])
plt.subplot(1,2,2)
plt.imshow(y[i])

# Augmentor test

In [ ]:
from albumentations import *

In [ ]:
x, y = make_xy_from_data_paths(train_x_paths[0:10],
                               train_y_paths[0:10],
                               image_size,
                               label,
                               'polygon',
                               'crop')

In [ ]:
i=0
#aug=RandomCrop(p=1, height=300, width=300)
#aug=ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.2, rotate_limit=45, p=1)
aug=HueSaturationValue(p=1)
auged = aug(image=x[i,:,:,:],mask=y[i,:,:,:])
img = auged["image"]
mask = auged["mask"]
mask = convert_y_to_image_array(mask[np.newaxis,:,:,:], image_size, label)
plt.figure(figsize=(30,10))
plt.subplot(1,3,1)
plt.imshow(x[i,:,:,:])
plt.subplot(1,3,2)
plt.imshow(img)
plt.subplot(1,3,3)
plt.imshow(mask[0])